# Supplementary videos

In [ ]:
import sys
sys.path.append("..")

from zebrafish import *

from scipy.ndimage import zoom

# Supplementary video 1

The gifs of neuronal activity within the ellipsoid were generated individually in the `Supp-Dynamics.ipynb` notebook, then assembled together using external software.

# Supplementary video 2

Multiplane calcium imaging mosaic video.

In [ ]:
top_directory = '/media/anleg84/KINGSTON/Datasets/Geometry/Reprocess/'

folders = identify_folders(top_directory, ['920nm'])

In [ ]:
folder = folders[2]

#### Saving filtered, downsampled, and mean-subtracted videos, then generating mosaic

In [ ]:
for folder in folders:
    
    files = identify_files(folder, ['corrected_', '.tif'])
    
    for file in tqdm(files):
        stack = load_stack(folder + file)
        stack = zoom(stack, (0.5, 0.5, 0.5), order=1)
        filtered = gaussian_filter(stack.astype('float'), (2, 1, 1))
        avg_frame = np.mean(filtered, axis=0)
        mean_subtracted = np.copy(filtered)
        for i in range(filtered.shape[0]):
            mean_subtracted[i] = filtered[i] - avg_frame
        mean_subtracted -= zero
        mean_subtracted[mean_subtracted < 0] = 0
        mean_subtracted[mean_subtracted > 65535] = 65535
        save_stack(folder + 'filtered_' + file, mean_subtracted.astype('uint16'))

    frames = []
    flag = True
    
    for i in tqdm(range(600), file=sys.stdout):
    
        volume = load_volumes(folder, [i], keywords=['filtered_', '.tif'])
        row, rows = [], []
        for j in range(5):
            row.append(volume[0][j])
        rows.append(np.concatenate(row, axis=1))
        row = []
        for j in range(5, 10):
            row.append(volume[0][j])
        rows.append(np.concatenate(row, axis=1))
        row = []
        for j in range(10, 15):
            row.append(volume[0][j])
        rows.append(np.concatenate(row, axis=1))
        
        frame = (255 * np.concatenate(rows, axis=0).astype('float')) / 65535
        saturation = 50
        frame[frame > saturation] = saturation # Adjusting contrast
        frame *= (255 / saturation)
        
        if flag:
            flag = False
            shape = frame.shape
            fourcc = cv2.VideoWriter_fourcc(*'MJPG')
            writer = cv2.VideoWriter(folder + 'mosaic.avi', fourcc, 50.0, (shape[1], shape[0]), False)
    
        writer.write(frame.astype('uint8'))

    writer.release()

    files = identify_files(folder, ['filtered_', '.tif'])
    for file in files:
        os.remove(folder + file)